In [1]:
!pip install sentence-transformers

     |████████████████████████████████| 55 kB 1.4 MB/s 
     |████████████████████████████████| 641 kB 8.0 MB/s 
     |████████████████████████████████| 3.8 MB 57.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-py3-none-any.whl size=74030 sha256=c934ca7ec73a2ebebc4a73b10116193b0fe123e1de18a2c64a7c503f7d70fbf9
  Stored in directory: /root/.cache/pip/wheels/3d/9c/3a/c98f3de96eba8d8d2d51842b8f156c53abca0a0a02ae1ba840
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2
  Attempting uninstall: transformers
    Found existing installation: transformers 2.7.0
    Uninstalling transformers-2.7.0:
      Successfully uninstalled transformers-2.7.0


In [2]:
import os
import re  # For preprocessing
import en_core_web_sm
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import pickle
from time import time  # To time our operations
import glob
import json
import zipfile
from tqdm import tqdm
import multiprocessing
import scipy

import torch
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
from sentence_transformers import SentenceTransformer
from typing import List, Dict
from sklearn.metrics.pairwise import cosine_similarity #for cosine similarity
from gensim.models.phrases import Phrases, Phraser #For create relevant phrases
from gensim.models import Word2Vec #Our model type


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [5]:
root_path = '/kaggle/input/CORD-19-research-challenge'

metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str,
    'doi': str
})


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
meta_df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [7]:
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)


In [8]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            if 'abstract' in content:
                for entry in content['abstract']:
                    self.abstract.append(entry['text'])
            else:
                self.abstract.append('Not provided.')
            # Body text
            if 'body_text' in content:
                for entry in content['body_text']:
                    self.body_text.append(entry['text'])
            else:
                self.body_text.append('Not provided.')
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)


    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'


In [9]:
def get_date_dt(all_json, meta_df):
    dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [],
             'abstract_summary': []}
    for idx, entry in tqdm(enumerate(all_json), desc="Parsing the articles Json's content", total=len(all_json)):
        content = FileReader(entry)

        # get metadata information
        meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
        # no metadata, skip this paper
        if len(meta_data) == 0:
            continue

        dict_['paper_id'].append(content.paper_id)
        dict_['abstract'].append(content.abstract)
        dict_['body_text'].append(content.body_text)
        
        # get metadata information
        meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]

        try:
            # if more than one author
            authors = meta_data['authors'].values[0].split(';')
            if len(authors) > 2:
                # more than 2 authors, may be problem when plotting, so take first 2 append with ...
                dict_['authors'].append(". ".join(authors[:2]) + "...")
            else:
                # authors will fit in plot
                dict_['authors'].append(". ".join(authors))
        except Exception as e:
            # if only one author - or Null valie
            dict_['authors'].append(meta_data['authors'].values[0])

        # add the title information, add breaks when needed
        dict_['title'].append(meta_data['title'].values[0])

        # add the journal information
        dict_['journal'].append(meta_data['journal'].values[0])
    return pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal'])

In [10]:
def Initialization_Word2Vec_model():
    w2v_model = Word2Vec(min_count=50,
                     window=10,
                     size=200,
                     sample=6e-5,
                     alpha=0.02,
                     min_alpha=0.0003,
                     negative=20,
                     workers=multiprocessing.cpu_count() -1)
    return w2v_model


In [11]:
def create_sentences(df, col_name):
    """Build sentences to Word2Vec model """
    t = time()
    sent = [row.split() for row in df[col_name]]
    phrases = Phrases(sent, min_count=50, progress_per=100, max_vocab_size=1000000)
    bigram = Phraser(phrases)

    print('Time to create sentences: {} mins'.format(round((time() - t) / 60, 2)))
    return bigram, bigram[sent]

def Build_Word2Vec_vocab(w2v_model, sentences,update=True):
    t = time()
    w2v_model.build_vocab(sentences, progress_per=10000, update=update)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    return w2v_model

def Train_Word2Vec_model(w2v_model, sentences):
    t = time()
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
    return w2v_model

def cleaning(spacy_doc):
    """Lemmatizes and removes stopwords
    doc needs to be a spacy Doc object """

    txt = [t.lemma_ for t in spacy_doc if
            t.dep_ not in ['prep', 'punct', 'det'] and
            len(t.text.strip()) > 2 and
            t.lemma_ != "-PRON-" and
            not t.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)
    else:
        return "no text"

def word2vec_preprocessing(df, column, offline=True) -> pd.DataFrame:
    # Prepare column from pd.DataFrame with text to Word2Vec model 
    url_pattern = r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
    data = df.drop_duplicates(column)
    data = data[
        data[column].apply(lambda x: len(x) > 0) &
        data[column].apply(lambda x: len(x.split()) > 3)
        ]
    data.reset_index(inplace=True, drop=True)
    data[column] = data[column].map(lambda x: re.sub(url_pattern, ' ', str(x)))
    brief_cleaning = (re.sub(r"[^a-zA-Z']+", ' ', str(row)).lower() for row in data[column])
    nlp = en_core_web_sm.load(disable=['ner', 'parser', 'tagger'])  # disabling Named Entity Recognition for speed
    t = time()
    txt = []
    if offline:
        paper_ids = data['paper_id']
        index = []
        for idx, doc in enumerate(nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1, n_process=5)):
            txt.append(cleaning(doc))
            index.append(paper_ids[idx]) #TODO verify its ok
        print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))
    else:
        for doc in nlp.pipe(brief_cleaning):
            txt.append(cleaning(doc))

    df_clean = data
    print([t for t in txt if t is None])
    df_clean[f'clean_{column}'] = txt
    return df_clean


In [12]:
def preprocessing(corpus: List, is_offline: bool):
    dt_corpus = pd.DataFrame({'clean': corpus})
    procesed = word2vec_preprocessing(dt_corpus, 'clean', offline=is_offline)
    return procesed['clean_clean'].to_list()

def get_relevant_articles(query,  articles_abstract):
    queries: List[str] = preprocessing([query], is_offline=False)
    articles_abstract.score = articles_abstract.score.astype(float) # to make the scores as float in order to not lose precision
    for q in queries:
        query_key_words: List[str] = get_unk_kw(phraser, q)
        embedded_query = embed_sentence(w2v_model, query_key_words)
        for idx, abstract in tqdm(articles_abstract.iterrows(), desc="Iterate over all articles abstract", total=len(articles_abstract)):
            abs_score = get_abstract_score(w2v_model, embedded_query, abstract['clean_abstract'])
            articles_abstract.at[idx, 'score'] = abs_score
    scored_articles_abstract = articles_abstract.sort_values('score', ascending=False)
    return scored_articles_abstract

Loading the data

In [13]:
data_dt = get_date_dt(all_json, meta_df)

Parsing the articles Json's content:  93%|█████████▎| 95694/103314 [30:08<03:21, 37.83it/s]

We will load our pretrained w2v model which was trained in this notebook earlier on.

In [14]:
to_train = False
if to_train:
    w2v_model = Initialization_Word2Vec_model()
    df_clean = word2vec_preprocessing(data_dt, 'body_text')
    phraser, sentences = create_sentences(df_clean, 'clean_body_text')
    update_w2v_vocab = len(w2v_model.wv.vocab) != 0
    w2v_model = Build_Word2Vec_vocab(w2v_model, sentences, update_w2v_vocab)
    w2v_model = Train_Word2Vec_model(w2v_model, sentences)
    # w2v_model.save('saved_model/w2v_model_on_all_abstract_full_text.w2v')
    w2v_model.init_sims(replace=True)
else:
    with open("/kaggle/input/w2v-model/saved_model/cleaned_to_w2v_all_document.pkl", 'rb') as f:
        df_clean = pickle.load(f)
    with open("/kaggle/input/w2v-model/phraser.pkl", 'rb') as f:
        phraser = pickle.load(f)
    w2v_model = Word2Vec.load('/kaggle/input/w2v-model/saved_model/w2v_model_on_all_abstract_full_text.w2v')

Now lets check that the w2v model make sense:
1. First we will see what it gices as the most similar words to a drig named Kaltera
2. We word see what it gives as the most similar word to bat
3. We will chec some sematic word calculation such as dead and country


In [15]:
print("most similar words to kaletra:")
for s_w in w2v_model.wv.most_similar(positive=[ 'kaletra']):
    print(s_w)
print('#'*100)
print("most similar words to bat:")
for s_w in w2v_model.wv.most_similar(positive=['bat']):
    print(s_w)
print('#'*100)
print("most similar words to dead and people:")
for s_w in w2v_model.wv.most_similar(positive=['dead', 'people']):
    print(s_w)

most similar words to kaletra:
('lopinavir_ritonavir', 0.7929786443710327)
('lopinavir', 0.7838401794433594)
('ritonavir', 0.7334166765213013)
('lpv', 0.704046368598938)
('darunavir', 0.6810516715049744)
('nelfinavir', 0.6048558354377747)
('saquinavir', 0.6048128008842468)
('atazanavir', 0.5886197686195374)
('zidovudine', 0.5843208432197571)
('ribavirin', 0.5715928673744202)
####################################################################################################
most similar words to bat:
('insectivorous', 0.7917876243591309)
('rhinolophid', 0.7473968267440796)
('schreibersii', 0.7356938719749451)
('perspicillata', 0.7316079139709473)
('miniopterus_schreibersii', 0.7254236936569214)
('artibeus_jamaicensis', 0.7246122360229492)
('microbats', 0.7197651863098145)
('vespertilionid', 0.7186751961708069)
('helvum', 0.7149186134338379)
('rousettus', 0.7127234935760498)
####################################################################################################
most similar

write something that show its a great results

In [16]:
ALPHA = 0.5

def embed_sentence(model, tokens: List[str]) -> List:
    res = []
    for t in tokens:
        try:
            vec = model.wv.word_vec(t, use_norm=False)
            res.append(vec)
        except KeyError:
            # logging.debug(f'Unidentified word while embedding:{t}')
            continue
    return res


def get_all_abstracts(phraser, cleaned_df):
    column = 'abstract'
    df = word2vec_preprocessing(cleaned_df, column, offline=True)
    df[f'clean_{column}'] = df[f'clean_{column}'].map(lambda x: get_unk_kw(phraser, x))
    df['score'] = [0] * len(df)
    return df


def get_unk_kw(phraser, query):
    if type(query) == str:
        return list(set(phraser[query.split()]))
    else:
        return []


def preprocessing(corpus: List, is_offline: bool):
    dt_corpus = pd.DataFrame({'clean': corpus})
    procesed = word2vec_preprocessing(dt_corpus, 'clean', offline=is_offline)
    return procesed['clean_clean'].to_list()


def get_abstract_score(model: Word2Vec, embedded_query:List, abstract: List[str]) -> float:
    f_score = 0
    valid_scores_sum = 1  # so we wouldn't divide by zero
    if type(abstract) != list or len(abstract) == 0:
        return f_score
    embedded_abstract = embed_sentence(model, abstract)
    for q_t in embedded_query:
        scores = model.wv.cosine_similarities(q_t, embedded_abstract)
        valid_scores = [s for s in scores if s > ALPHA]
        valid_scores_sum += len(valid_scores)
        f_score += np.sum(valid_scores)
    norm_score = f_score / valid_scores_sum # to normalize by the number of tokens that was counted
    return norm_score


In [18]:
print(w2v_model.wv.most_similar(positive=['medicine', 'covid'], negative=['sars']))
print(w2v_model.wv.most_similar(positive=['dead', 'covid','israel']))
print(w2v_model.wv.similarity('kaletra', 'covid'))
print(w2v_model.wv.similarity('kaletra', 'sars'))



[('specialty', 0.5728989243507385), ('medical', 0.5694969892501831), ('hepatology', 0.5282034873962402), ('subspecialty', 0.5248998403549194), ('cardiology', 0.5120755434036255), ('practitioner', 0.5110015869140625), ('psychiatry', 0.507890522480011), ('herbalist', 0.5011195540428162), ('oncology', 0.4991220533847809), ('personalise', 0.4916495084762573)]
[('iran', 0.5408326983451843), ('italy', 0.5315767526626587), ('february', 0.5230684280395508), ('march', 0.5145354270935059), ('sick', 0.5124857425689697), ('spain', 0.5029584169387817), ('die', 0.49855750799179077), ('israeli', 0.49411678314208984), ('case', 0.49138104915618896), ('china', 0.4906347393989563)]
0.2817064
0.31015366


And than we load it again

In [19]:
with open('/kaggle/input/w2v-model/parsed_abstract_ran_offline.pk', 'rb') as f: 
    articles_abstract = pickle.load(f)


In [20]:
articles_abstract['body_text'].head()

0    VP3, and VP0 (which is further processed to VP...
1    In December 2019, a novel coronavirus, SARS-Co...
2    Metagenomic sequencing, which allows us to dir...
3    Infectious bronchitis (IB), which is caused by...
4    Nipah is an infectious negative-sense single-s...
Name: body_text, dtype: object

In [21]:
encoder = SentenceTransformer("roberta-large-nli-stsb-mean-tokens")

100%|██████████| 1.31G/1.31G [03:01<00:00, 7.22MB/s]


In [22]:
encoded_articles_abstract = encoder.encode(articles_abstract['abstract'].tolist())
articles_abstract['encoded_articles_abstract'] = encoded_articles_abstract    

Batches: 100%|██████████| 3277/3277 [08:35<00:00,  6.35it/s]


In [23]:
def query_questions(query, articles_abstract):    
    encoded_query = encoder.encode([query])
    articles_abstract['distances'] = scipy.spatial.distance.cdist(encoded_query, articles_abstract['encoded_articles_abstract'].tolist(), "cosine")[0]
    articles_abstract = articles_abstract.sort_values('distances').reset_index()[:70]
    
    articles_abstract['sentence_list'] = [body.split(". ") for body in articles_abstract['body_text'].to_list()] 
    paragraphs = []
    for index, ra in articles_abstract.iterrows():
        para_to_add = [". ".join(ra['sentence_list'][n:n+7]) for n in range(0, len(ra['sentence_list']), 7)]        
        para_to_add.append(ra['abstract'])
        paragraphs.append(para_to_add)
    articles_abstract['paragraphs'] = paragraphs
    answers = answer_question(query, articles_abstract)
    return answers

In [24]:
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import pipeline

def get_QA_bert_model():
    torch_device = 0 if torch.cuda.is_available() else -1
    tokenizer = AutoTokenizer.from_pretrained("ahotrod/albert_xxlargev1_squad2_512")
    model = AutoModelForQuestionAnswering.from_pretrained("ahotrod/albert_xxlargev1_squad2_512")
    return pipeline('question-answering', model=model, tokenizer=tokenizer,device=torch_device)

In [ ]:
nlp_qa = get_QA_bert_model()

In [26]:
def answer_question(question: str, context_list):
    # anser question given question and context
    answers =[]
    all_para = [item for sublist in context_list['paragraphs'].to_list() for item in sublist] 
    print(f"paragraph to scan: {len(all_para)}")
    for _, article in context_list.iterrows():
        for context in article['paragraphs']:
            if len(context) < 10:
                continue
            with torch.no_grad():
                answer = nlp_qa(question=question, context=context)
            answer['paragraph'] = context
            answer['paper_id'] = article['paper_id']
            answers.append(answer)            
    df = pd.DataFrame(answers)
    df = df.sort_values(by='score', ascending=False)
    return df



> Doing some sanity check: 
Looking for 20 Q&A manually tagged in the article corpus:


In [27]:
answer1 = query_questions("Are there geographic variations in the rate of COVID-19 spread?", articles_abstract)
answer1.to_csv("/kaggle/working/1_Are there geographic variations in the rate of COVID-19 spread.csv")
answer1.head()

Batches: 100%|██████████| 1/1 [00:00<00:00, 30.51it/s]


paragraph to scan: 1152


,score,start,end,answer,paragraph,paper_id
764,0.950877,881,952,Heterogeneity in growth laws across the differ...,Some examples of such results are presented in...,a7d9440cb3a45994c274d4de41cbaf14343c8a89
594,0.898456,594,649,uniform restrictions were imposed all over the...,"Hopefully, these measures affect the spread of...",5ec284329dfa97be65ed5e598c40426f03499186
176,0.883789,274,303,vary widely between countries,1101 Mortality : A major policy concern is tha...,d10ff5d67fb2f98d197072a87f75f1b5ca773491
1054,0.854489,403,447,amplified transmission in confined settings.,Although Singapore has been detecting and isol...,b842a645116edb86df997aa960699a7513f2196a
443,0.838204,1406,1485,we assumed that the transmission dynamics in e...,The transmission dynamics are described by the...,3599c933334227fd95014d36baa4b87e41ffd1e1


In [28]:
answer2 = query_questions("What works have been done on infection spreading?", articles_abstract)
answer2.to_csv("/kaggle/working/2_What works have been done on infection spreading.csv")
answer2.head()

Batches: 100%|██████████| 1/1 [00:00<00:00, 40.25it/s]


paragraph to scan: 1856


,score,start,end,answer,paragraph,paper_id
1118,0.801475,749,759,genotyping,A further six cases were identified by matchin...,29e0c4a7f3f8e2bd4a8f50fae12cc31f1a863763
1085,0.739059,1188,1208,advances in virology,The last natural case occurred in Somalia in 1...,29e0c4a7f3f8e2bd4a8f50fae12cc31f1a863763
894,0.646645,774,797,Outbreak investigations,The total number of cases of a particular dise...,fe27017f9e0a5c74edc93d2a2d087110e89de14c
791,0.626517,640,686,Inhibitors of viral fusion have been developed,"When these Δ-peptides from EBOV, SUDV, and TAF...",c75994b5485339cdf4180bedab58e60366e49740
1039,0.601572,677,694,(Pornillos et al.,2001) . TSG101 is involved in cellular transcr...,f3042820ac417ec01ca0e15a4232e24b8d78ba74


In [29]:
answer3 =  query_questions("Are there geographic variations in the mortality rate of COVID-19?", articles_abstract)
answer3.to_csv("/kaggle/working/3_Are there geographic variations in the mortality rate of COVID.csv")
answer3.head()

Batches: 100%|██████████| 1/1 [00:00<00:00, 25.36it/s]


paragraph to scan: 1167


,score,start,end,answer,paragraph,paper_id
530,0.922197,274,303,vary widely between countries,1101 Mortality : A major policy concern is tha...,d10ff5d67fb2f98d197072a87f75f1b5ca773491
1127,0.764439,514,557,All the cases were from hospitals in China.,"A higher risk of with hypertension, respirator...",4090d8537051fae501355bc567dc2dec620762eb
857,0.717689,775,825,Most studies reported COVID-19 patients from C...,All analyses were conducted with STATA 14 or 1...,58869cc8053741f55e3db6e1d45e758a79e3fb3f
985,0.713183,0,48,"Some countries also appear high mortality rates,",Some countries also appear high mortality rate...,d57b13c4de6f1c057e1fc3025dcfe5e141b8ca9a
256,0.688992,829,851,crosscountry variation,medRxiv preprint 2 Background: The population-...,d906c6a23278fa3514ae0ba8918241cb1afbc02e


In [30]:
answer4 =  query_questions("Is there any evidence to suggest geographic based virus mutations?", articles_abstract)
answer4.to_csv("/kaggle/working/4_Is there any evidence to suggest geographic based virus mutations.csv")
answer4.head()

Batches: 100%|██████████| 1/1 [00:00<00:00, 22.72it/s]


paragraph to scan: 1970


,score,start,end,answer,paragraph,paper_id
1631,0.859699,1508,1572,sampled bat populations are likely to be geogr...,Patterns of association/disassociation were ex...,634128ea7d7736750e1c3cd0a48bb37843d06dac
430,0.836274,422,495,elevation appears to impede the movement of ra...,We generated 100 artificial data sets for each...,e3c22f7e1409db69c1be9a7ae76319405506aaea
50,0.819915,1159,1198,heterogeneity of MERS-CoV among patients,The deletion resulted in the production of a s...,01880ef487dfed3681dd6d296a0a82417b8f38a0
1912,0.804085,570,647,cluster 6 does not seem to be associated with ...,Cluster number 1 is formed by episomes that ha...,f3b3d8711d03bd36a685c282b7a83a37f874fd9c
1029,0.742012,698,755,geographical clusters with high predicted host...,"3) , with similar hotspots when geographical d...",fae708f3b0a6023fccad707c4417b521f84fc328
